<a href="https://colab.research.google.com/github/kdfernander/Miami-Airbnb-Geospatial-Trends/blob/main/geospatial_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap

df=pd.read_csv('/content/drive/MyDrive/miami_airbnb_data.csv')
df.head()


,Listing Title,Property Type,Listing Type,Created Date,Last Scraped Date,Last Host Count Updated Date,Country,State,City,Zipcode,...,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Number of Bookings LTM - Number of observed month,Average Daily Rate (USD),Average Daily Rate (Native),Annual Revenue LTM (USD),Annual Revenue LTM (Native)
0,Cozy Family home with a pool,Entire home,entire_home,2023-07-31,2024-01-03,NaN,US,NaN,Miami,33138,...,84,281,6,9,0,-3,222,222,1678,1678
1,Coconut Grove Cottage in the Heart of the Vill...,Entire guesthouse,entire_home,2023-07-31,2023-10-11,NaN,US,Florida,Miami,33133,...,178,187,83,56,21,18,94,94,11470,11470
2,DOWNTOWN MODERN INCREDIBLE VIEWS 2/2 LOFT,Entire rental unit,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33132,...,92,273,70,91,15,12,291,291,25863,25863
3,"WELCOME TO PARADISE MIAMI, FLORIDA",Private room in rental unit,private_room,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33129,...,210,155,168,96,37,34,60,60,12096,12096
4,Garden Cottage- Heart of Design District,Entire guesthouse,entire_home,2023-07-31,2024-01-03,NaN,US,Florida,Miami,33137,...,53,312,39,88,2,-1,52,52,2575,2575


In [3]:
df.columns

Index(['Listing Title', 'Property Type', 'Listing Type', 'Created Date',
       'Last Scraped Date', 'Last Host Count Updated Date', 'Country', 'State',
       'City', 'Zipcode', 'Neighbourhood', 'Metropolitan Statistical Area',
       'Currency Native', 'Number of Reviews', 'Bedrooms', 'Bathrooms',
       'Max Guests', 'Airbnb Superhost', 'Cancellation Policy',
       'Cleaning Fee (USD)', 'Cleaning Fee (Native)', 'Extra People Fee (USD)',
       'Extra People Fee(Native)', 'Check-in Time', 'Checkout Time',
       'Minimum Stay', 'Latitude', 'Longitude', 'Exact Location',
       'Overall Rating', 'Airbnb Communication Rating',
       'Airbnb Accuracy Rating', 'Airbnb Cleanliness Rating',
       'Airbnb Checkin Rating', 'Airbnb Location Rating',
       'Airbnb Value Rating', 'Amenities', 'picture_url', 'License',
       'Airbnb Property ID', 'Airbnb Host ID', 'Host Listing Count',
       'guest_controls', 'instant_bookable', 'Pets Allowed', 'Listing URL',
       'Instantbook Enabled', 

In [4]:
#we are only keeping the following columns since this geospatial analysis will be based on seasonal trends and bookings:
col_to_keep = [
    'Latitude', 'Longitude', 'Zipcode', 'Neighbourhood', 'City',
    'Count Available Days LTM', 'Count Blocked Days LTM', 'Count Reservation Days LTM',
    'Occupancy Rate LTM', 'Number of Bookings LTM', 'Average Daily Rate (USD)',
    'Annual Revenue LTM (USD)', 'Property Type', 'Listing Type', 'Overall Rating', 'Amenities'
]

df=df[col_to_keep]
df.columns

Index(['Latitude', 'Longitude', 'Zipcode', 'Neighbourhood', 'City',
       'Count Available Days LTM', 'Count Blocked Days LTM',
       'Count Reservation Days LTM', 'Occupancy Rate LTM',
       'Number of Bookings LTM', 'Average Daily Rate (USD)',
       'Annual Revenue LTM (USD)', 'Property Type', 'Listing Type',
       'Overall Rating', 'Amenities'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10205 entries, 0 to 10204
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Latitude                    10205 non-null  float64
 1   Longitude                   10205 non-null  float64
 2   Zipcode                     10205 non-null  int64  
 3   Neighbourhood               3485 non-null   object 
 4   City                        10205 non-null  object 
 5   Count Available Days LTM    10205 non-null  int64  
 6   Count Blocked Days LTM      10205 non-null  int64  
 7   Count Reservation Days LTM  10205 non-null  int64  
 8   Occupancy Rate LTM          10205 non-null  int64  
 9   Number of Bookings LTM      10205 non-null  int64  
 10  Average Daily Rate (USD)    10205 non-null  int64  
 11  Annual Revenue LTM (USD)    10205 non-null  int64  
 12  Property Type               10205 non-null  object 
 13  Listing Type                102

In [6]:
df.head()

,Latitude,Longitude,Zipcode,Neighbourhood,City,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Average Daily Rate (USD),Annual Revenue LTM (USD),Property Type,Listing Type,Overall Rating,Amenities
0,25.84454,-80.18748,33138,NaN,Miami,84,281,6,9,0,222,1678,Entire home,entire_home,5.0,"['Free parking on premises', 'Kitchen', 'Wifi'..."
1,25.72951,-80.24454,33133,Coconut Grove,Miami,178,187,83,56,21,94,11470,Entire guesthouse,entire_home,5.0,"['Kitchen', 'Wifi', 'Dedicated workspace', 'Dr..."
2,25.77700,-80.19000,33132,Miami Government District,Miami,92,273,70,91,15,291,25863,Entire rental unit,entire_home,4.5,"['Free parking on premises', 'Elevator', 'Kitc..."
3,25.76100,-80.20300,33129,Brickell,Miami,210,155,168,96,37,60,12096,Private room in rental unit,private_room,5.0,"['Free parking on premises', 'Kitchen', 'Wifi'..."
4,25.81790,-80.19133,33137,Design District,Miami,53,312,39,88,2,52,2575,Entire guesthouse,entire_home,5.0,"['Kitchen', 'Wifi', 'Dedicated workspace', 'Dr..."


In [7]:
df.isnull().sum()

Latitude                         0
Longitude                        0
Zipcode                          0
Neighbourhood                 6720
City                             0
Count Available Days LTM         0
Count Blocked Days LTM           0
Count Reservation Days LTM       0
Occupancy Rate LTM               0
Number of Bookings LTM           0
Average Daily Rate (USD)         0
Annual Revenue LTM (USD)         0
Property Type                    0
Listing Type                     0
Overall Rating                2523
Amenities                        0
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
df

,Latitude,Longitude,Zipcode,Neighbourhood,City,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Average Daily Rate (USD),Annual Revenue LTM (USD),Property Type,Listing Type,Overall Rating,Amenities
1,25.72951,-80.24454,33133,Coconut Grove,Miami,178,187,83,56,21,94,11470,Entire guesthouse,entire_home,5.0,"['Kitchen', 'Wifi', 'Dedicated workspace', 'Dr..."
2,25.77700,-80.19000,33132,Miami Government District,Miami,92,273,70,91,15,291,25863,Entire rental unit,entire_home,4.5,"['Free parking on premises', 'Elevator', 'Kitc..."
3,25.76100,-80.20300,33129,Brickell,Miami,210,155,168,96,37,60,12096,Private room in rental unit,private_room,5.0,"['Free parking on premises', 'Kitchen', 'Wifi'..."
4,25.81790,-80.19133,33137,Design District,Miami,53,312,39,88,2,52,2575,Entire guesthouse,entire_home,5.0,"['Kitchen', 'Wifi', 'Dedicated workspace', 'Dr..."
5,25.77652,-80.18919,33132,Miami Government District,Miami,61,304,3,6,0,125,458,Entire condo,entire_home,5.0,"['Free parking on premises', 'Elevator', 'Kitc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4871,25.76067,-80.18913,33131,Miami Financial District,Miami,3,362,1,40,0,185,332,Entire condo,entire_home,5.0,"['Kitchen', 'Wifi', 'Hot tub', 'Pool', 'TV', '..."
4872,25.81472,-80.19063,33137,Design District,Miami,341,24,181,64,36,188,46429,Entire condo,entire_home,5.0,"['Free parking on premises', 'Elevator', 'Kitc..."
4875,25.76977,-80.18780,33131,Brickell Key,Miami,345,20,212,74,56,240,68272,Entire condo,entire_home,5.0,"['Elevator', 'Kitchen', 'Wifi', 'Gym', 'Hot tu..."
4877,25.83227,-80.21655,33150,Model City,Miami,156,209,51,39,8,84,5111,Entire guest suite,entire_home,5.0,"['Free parking on premises', 'Wifi', 'Iron', '..."


In [10]:
round(df.describe())

,Latitude,Longitude,Zipcode,Count Available Days LTM,Count Blocked Days LTM,Count Reservation Days LTM,Occupancy Rate LTM,Number of Bookings LTM,Average Daily Rate (USD),Annual Revenue LTM (USD),Overall Rating
count,3281.0,3281.0,3281.0,3281.0,3281.0,3281.0,3281.0,3281.0,3281.0,3281.0,3281.0
mean,26.0,-80.0,33133.0,191.0,174.0,107.0,63.0,26.0,207.0,28425.0,5.0
std,0.0,0.0,5.0,132.0,132.0,87.0,23.0,28.0,254.0,32565.0,0.0
min,26.0,-80.0,33122.0,0.0,0.0,0.0,2.0,0.0,10.0,58.0,0.0
25%,26.0,-80.0,33130.0,61.0,42.0,25.0,47.0,4.0,103.0,6644.0,4.0
50%,26.0,-80.0,33131.0,201.0,164.0,94.0,64.0,19.0,154.0,18486.0,5.0
75%,26.0,-80.0,33135.0,323.0,304.0,178.0,81.0,40.0,241.0,39520.0,5.0
max,26.0,-80.0,33178.0,365.0,365.0,341.0,100.0,251.0,7392.0,518144.0,5.0


In [11]:
miami_map = folium.Map(location=[25.7617, -80.1918], zoom_start=12) #base map centred around miami with the zoom level starting at a balanced wideview

'''NOTE: NS position in the first param of location and EW position in the second param of location
was founded using google map to determine the postion of miami for the folio interactive tool'''


'NOTE: NS position in the first param of location and EW position in the second param of location\nwas founded using google map to determine the postion of miami for the folio interactive tool'

In [12]:
grouped_data_by_neighborhoods=df.groupby('Neighbourhood').agg({
    'Latitude':'mean',
    'Longitude':'mean',
    'Count Available Days LTM':'sum',
    'Count Blocked Days LTM':'sum',
    'Count Reservation Days LTM':'sum',
    'Occupancy Rate LTM':'mean',
    'Number of Bookings LTM':'sum',
    'Average Daily Rate (USD)':'mean',
    'Annual Revenue LTM (USD)':'sum'
    }).reset_index() #to turn it back into a regular df for feasibility


In [13]:
#creating heatmap for availability
heat_map = [
    [row['Latitude'], row['Longitude'], row['Count Available Days LTM']]
    for index, row in grouped_data_by_neighborhoods.iterrows()
    if not pd.isnull(row['Latitude']) and not pd.isnull(row['Longitude'])
] #for each index, extracts/selects the col specified in the param in the ds row
  #filters out null values within lat and long if any available

HeatMap(heat_map, radius=10, max_zoom=13).add_to(miami_map)

In [15]:
#adding markers for neighborhoods with info on bookings
for index, row in grouped_data_by_neighborhoods.iterrows():
    if not pd.isnull(row['Latitude']) and not pd.isnull(row['Longitude']):
        popup_text = f"""
        Neighborhood: {row['Neighbourhood']}<br>
        Number of Bookings: {row['Number of Bookings LTM']}<br>
        Availability: {row['Count Available Days LTM']}<br>
        Average Daily Rate: ${row['Average Daily Rate (USD)']}<br>
        Annual Revenue: ${row['Annual Revenue LTM (USD)']}
        """
        folium.Marker( #for navigation
            location=[row['Latitude'], row['Longitude']],
            popup=popup_text,
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(miami_map)


In [16]:
output_file = 'miami_airbnb_map.html'
miami_map.save(output_file) #saving the map to a html file

In [17]:
output_file

'miami_airbnb_map.html'